<div style="text-align: center;">
<h1 style="font-style:oblique; color:red">SQL MURDER MYSTERY</h1>
<h3 style="margin-top:-25px">Can you find out whodunnit?</h3>
<img src="img/illustration.png" />
</div>


A crime has taken place and the detective needs your help. The detective gave you the crime scene report, but you somehow lost it. You vaguely remember that the crime was a **murder** that occurred sometime on **Jan.15, 2018** and that it took place in **SQL City**. Start by retrieving the corresponding crime scene report from the police department’s database.


### Modelo de datos
![sql_game_schemma.png](img/schema.png)

In [2]:
import pandas as pd
import sqlite3

def datos(query, ruta_db = "./data/sql-murder-mystery.db"):
    try:
        with sqlite3.connect(ruta_db) as connection:
            res = pd.read_sql(query, connection)
        return res
    except:
        print("Se ha producido un error con la conexión o la consulta.")
        return None

Primero, consultamos la tabla **crime_scene_report** para obtener la información inicial del crimen.

In [15]:
info = datos('''
    SELECT *
    FROM crime_scene_report
    WHERE date = "20180115" AND type = "murder" AND city = "SQL City"
    ''')
print(info["description"].iloc[0])

Security footage shows that there were 2 witnesses. The first witness lives at the last house on "Northwestern Dr". The second witness, named Annabel, lives somewhere on "Franklin Ave".


Con el resultado obtenido:  
> Security footage shows that there were 2 witnesses. The first witness lives at the last house on "Northwestern Dr". The second witness, named Annabel, lives somewhere on "Franklin Ave".  

Podemos continuar extrayendo más información sobre los testigos. Para ello, buscaremos en la tabla **person** para localizar quiénes son.

In [16]:
info = datos('''
    SELECT * 
    FROM person 
    WHERE address_street_name = "Northwestern Dr" 
    AND address_number = (SELECT MAX(address_number) FROM person WHERE address_street_name = "Northwestern Dr")
    ''')
info

,id,name,license_id,address_number,address_street_name,ssn
0,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949


In [18]:
info = datos('''
    SELECT *
    FROM person 
    WHERE name LIKE '%Annabel%' AND address_street_name = "Franklin Ave"
    ''')
info

,id,name,license_id,address_number,address_street_name,ssn
0,16371,Annabel Miller,490173,103,Franklin Ave,318771143


Ahora que ya tenemos el id y el nombre de los testigos, vamos a consultar la tabla **interview** para conseguir más información y saber por dónde seguir investigando.

In [19]:
info = datos('''
    SELECT * 
    FROM interview 
    WHERE person_id = 14887
    ''')
print(info["transcript"].iloc[0])

I heard a gunshot and then saw a man run out. He had a "Get Fit Now Gym" bag. The membership number on the bag started with "48Z". Only gold members have those bags. The man got into a car with a plate that included "H42W".


In [20]:
info = datos('''
    SELECT * 
    FROM interview 
    WHERE person_id = 16371
    ''')
print(info["transcript"].iloc[0])

I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th.


Con estas declaraciones, hemos obtenido datos muy valiosos para poder seguir investigando. Sabemos que:  

- El hombre que salió corriendo de la escena va al gimnasio *Get Fit Now Gym* y su número de socio empieza por *48Z*. Además, llevaba una mochila que sólo llevan los *gold members*. La matrícula de su coche incluía los caracteres *H42W*.
- Estaba en el gimnasio el 9 de enero.  

Primero, vamos a obtener los datos del gimnasio que coinciden con la declaración de los testigos. Para ello, necesitamos consultar la tabla **get_fit_now_member**

In [21]:
info = datos('''
    SELECT *
    FROM get_fit_now_member 
    WHERE id LIKE '48Z%' AND membership_status = "gold"
    ''')
info

,id,person_id,name,membership_start_date,membership_status
0,48Z7A,28819,Joe Germuska,20160305,gold
1,48Z55,67318,Jeremy Bowers,20160101,gold


Tenemos dos principales sospechosos que coinciden con la declaración de Morty Schapiro. Vamos a ver si podemos descartar alguno con la declaración de Annabel Miller, comprobando si algunos de ellos estuvo en el gimnasio en día 9 de enero. Para ello, consultamos la tabla **get_fit_now_check_in**

In [22]:
info = datos('''
    SELECT *
    FROM get_fit_now_check_in 
    WHERE check_in_date = "20180109" AND membership_id IN ("48Z7A", "48Z55")
    ''')
info

,membership_id,check_in_date,check_in_time,check_out_time
0,48Z7A,20180109,1600,1730
1,48Z55,20180109,1530,1700


Parece que los dos estuvieron ese día en el gimnasio, por tanto, no nos da mucha información extra. Tenemos que tirar de otra parte de la declaración de Morty Schapiro: *la matrícula del coche*.  

Vamos a buscar los datos del vehículo que contenía los caracteres de la matrícula dados en la declaración. Para ello, vamos a comprobar los vehículos de los dos sospechosos directamente, con las tablas **person** y **drivers_license**

In [25]:
info = datos('''
    SELECT p.id, p.name, dl.id, dl.plate_number
    FROM person p
    INNER JOIN drivers_license dl
    ON p.license_id = dl.id
    WHERE p.id IN (28819, 67318)
    ''')
info

,id,name,id,plate_number
0,67318,Jeremy Bowers,423327,0H42W2


Hemos obtenido una información que puede ser clave. Observamos que la matrícula del vehículo de uno de los sospechosos coincide con las declaraciones del testigo.  

Podría ser una pista definitiva, pero vamos a realizar la última consulta para asegurarnos. Veamos qué nos dice la tabla de **facebook_event_checkin**. Y vamos a utilizar tanto la información de los testigos como de los sospechosos, para encontrar coincidencias o pistas relevantes.

In [28]:
info = datos('''
    SELECT f.*, p.name
    FROM facebook_event_checkin f
    INNER JOIN person p
    ON f.person_id = p.id
    WHERE p.id IN (14887, 16371, 28819, 67318)
    ''')
info

,person_id,event_id,event_name,date,name
0,14887,4719,The Funky Grooves Tour,20180115,Morty Schapiro
1,16371,4719,The Funky Grooves Tour,20180115,Annabel Miller
2,67318,4719,The Funky Grooves Tour,20180115,Jeremy Bowers
3,67318,1143,SQL Symphony Concert,20171206,Jeremy Bowers


¡AJÁ! Acabamos de comprobar que el día del asesinato, tanto los dos testigos como el principal sospechoso coincidieron en el evento *The Funky Grooves Tour*.  

Por tanto, todas las pistas y declaraciones nos llevan a la conclusión de que...  

# Jeremy Bowers es el asesino.  
(bueno, presunto...)